---

Here use you CSV from WIX: Example! "Pole Studios (2).csv", replace the Name of the CSV inside the "", Run All Cells!

In [34]:
import pandas as pd

# Load initial URLs
hop_dataframe = pd.read_csv("Pole Studios (2).csv")
hop_dataframe.head(3)

,Pole Studio,PLZ,Stadt,Land,Adresse,Telefonnummer,E-Mail-Adresse,Webseite,Facebook,Instagram,...,ID,Owner,Created Date,Updated Date,Bildergalerie,Richtigkeit der Angaben,Pole Studios (Pole Studio),Online Studios (Pole Studio),Inaktiv,Löschvermerk
0,Seemannsbraut Poledance Flensburg,24941,Flensburg,Deutschland,"{""subdivisions"":[{""code"":""SH"",""name"":""Schleswi...",15163300769,seemannsbraut-poledance@web.de,https://www.seemannsbraut-poledance.de/,https://www.facebook.com/SeemannsbrautPoledance,https://www.instagram.com/seemannsbraut_poleda...,...,01df35bd-209a-4520-a7bc-d109b9aaf2f6,284241e1-0e53-4183-b67f-34edcfb7c8b1,2022-11-18T11:57:31Z,2024-02-04T18:00:48Z,NaN,NaN,/polestudio/seemannsbraut-poledance-flensburg,/onlinestudio/seemannsbraut-poledance-flensburg,NaN,NaN
1,TSG Blau Gold Ballett- und Pole-dance- Studio,35390,Gießen,Deutschland,"{""subdivisions"":[{""code"":""HE"",""name"":""Hessen"",...",0491 621606468,tanz-giessen@gmx.de,https://www.tsg-giessen.de/angebote/tanz/poled...,https://www.facebook.com/poledancegiessen.de,https://www.instagram.com/tsg_blaugoldgiessen....,...,0233997c-a277-4179-990a-347e1cea347e,284241e1-0e53-4183-b67f-34edcfb7c8b1,2022-11-18T11:57:31Z,2024-02-04T18:00:49Z,NaN,NaN,/polestudio/tsg-blau-gold-ballett--und-pole-da...,/onlinestudio/tsg-blau-gold-ballett--und-pole-...,NaN,NaN
2,Pole Stars Osnabrück,49080,Osnabrück,Deutschland,"{""subdivisions"":[{""code"":""NDS"",""name"":""Nieders...",01573 1810262,info@polestars-osnabrueck.de,http://www.polestars-osnabrueck.de/,https://www.facebook.com/PoleStarsOsnabrueck.de/,https://www.instagram.com/pole_stars_osnabruec...,...,04673b9d-d8b2-4caf-8632-d349eab2cd4e,284241e1-0e53-4183-b67f-34edcfb7c8b1,2022-11-18T11:57:31Z,2024-02-04T18:00:48Z,NaN,NaN,/polestudio/pole-stars-osnabr%C3%BCck,/onlinestudio/pole-stars-osnabr%C3%BCck,NaN,NaN


In [35]:
hop_polestudios = hop_dataframe["Pole Studios (Pole Studio)"].to_list()

initial_urls = hop_polestudios

base_url = "https://www.eversports.de/s"

#initial_urls = list(set(hop_dataframe["Pole Studios (Pole Studio)"]))
len(initial_urls)

319

In [36]:
url = []
for url_part in initial_urls:
    url.append(base_url+url_part[11:])
url[:10]

['https://www.eversports.de/s/seemannsbraut-poledance-flensburg',
 'https://www.eversports.de/s/tsg-blau-gold-ballett--und-pole-dance--studio',
 'https://www.eversports.de/s/pole-stars-osnabr%C3%BCck',
 'https://www.eversports.de/s/aerial-art-studio',
 'https://www.eversports.de/s/k%C3%B6rpernah',
 'https://www.eversports.de/s/dance-moves-wolfenb%C3%BCttel',
 'https://www.eversports.de/s/mypole',
 'https://www.eversports.de/s/tanzstudio-michel',
 'https://www.eversports.de/s/pole-sensation',
 'https://www.eversports.de/s/poledance-rostock']

In [37]:
initial_urls = url

Test V7

In [38]:
import pandas as pd
from a_PyCaller import process_urls
from tqdm import tqdm
from datetime import datetime

def process_and_print_results(url, all_pole_studio_data, all_workshops_data, all_workshop_details_data):
    data = process_urls([url])

    if data:
        for key, df in data.items():
            if df is not None and not df.empty:
                tqdm.write(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - INFO - Scraping Data from: {url}")
                tqdm.write(f"{key.replace('_', ' ').title()}: {len(df)} entries")

                # Update the appropriate DataFrame
                if key == 'pole_studio_data':
                    all_pole_studio_data = pd.concat([all_pole_studio_data, df], ignore_index=True)
                elif key == 'workshops_data':
                    all_workshops_data = pd.concat([all_workshops_data, df], ignore_index=True)
                elif key == 'workshop_details':
                    all_workshop_details_data = pd.concat([all_workshop_details_data, df], ignore_index=True)

    return all_pole_studio_data, all_workshops_data, all_workshop_details_data

def main():
    # # Load initial URLs
    # all_found_urls_s = pd.read_csv("your_output_file.csv")
    # initial_urls = list(set(all_found_urls_s["0"])) # Delete "[:3]" to scrape all Urls

    # Initialize DataFrames
    all_pole_studio_data = pd.DataFrame()
    all_workshops_data = pd.DataFrame()
    all_workshop_details_data = pd.DataFrame()

    # Process each URL with tqdm
    with tqdm(initial_urls, desc="Processing URLs", dynamic_ncols=True) as pbar:
        for url in pbar:
            all_pole_studio_data, all_workshops_data, all_workshop_details_data = process_and_print_results(
                url, all_pole_studio_data, all_workshops_data, all_workshop_details_data
            )

    # # Export DataFrames to CSV files
    # all_pole_studio_data.to_csv("Pole_Studio_Übersicht_S.csv", index=False)
    # all_workshops_data.to_csv("Workshop_Liste_SW.csv", index=False)
    # all_workshop_details_data.to_csv("Workshop_Übersicht_E.csv", index=False)

    # Return the final DataFrames
    return all_pole_studio_data, all_workshops_data, all_workshop_details_data

# Run the main function and get the final DataFrames
result_pole_studio, result_workshops, result_workshop_details = main()


Processing URLs:   0%|          | 0/319 [00:00<?, ?it/s]

Starting URL reconstruction...


Processing URLs:   0%|          | 1/319 [00:01<06:26,  1.21s/it]

Error during HTTP request: 404 Client Error: Not Found for url: https://www.eversports.de/s/seemannsbraut-poledance-flensburg
Converting to DataFrame...
Validating URLs...
Processing completed.
Starting URL reconstruction...


Processing URLs:   1%|          | 2/319 [00:02<06:25,  1.22s/it]

...

Error during HTTP request: Exceeded 30 redirects.
Converting to DataFrame...
Validating URLs...
Processing completed.
Starting URL reconstruction...


Processing URLs:   2%|▏         | 7/319 [00:16<12:34,  2.42s/it]

Error during HTTP request: 404 Client Error: Not Found for url: https://www.eversports.de/s/mypole
Converting to DataFrame...
Validating URLs...
Processing completed.
Starting URL reconstruction...


Processing URLs:   2%|▏         | 7/319 [00:17<13:08,  2.53s/it]


KeyboardInterrupt: 

In [ ]:
result_pole_studio

,PoleStudio_Name,Adresse,PLZ,Stadt,Straße,Buttons,Pole Studio Beschreibung,E-Mail,Homepage,Telefon,URL_S,Art,Angebot,Created Date,Updated Date
0,Körperformfabrik GbR,"[Hauptstraße 108 , 77855 Achern]",77855,Achern,Hauptstraße 108,"[Übersicht, Klassen, Preise, Team]",Pole Dance oder Pylon Dance ist Körperkunst un...,alexander.malyschkin@web.de,http://www.koerperformfabrik.de,+49 17672823704,https://www.eversports.de/s/koerperformfabrik-gbr,"[TRX®, TRX® Yoga, Exotic, Poledance, Poledance...",Kostenlos Schnuppern,2024-02-10 16:20:45,2024-02-10 16:20:45
1,PoleworkX,"[Schleißheimer Straße 523, 80933 München]",80933,München,Schleißheimer Straße 523,[Übersicht],Komme ins PoleworkX München für Poledancetrain...,info@poleworkx.de,None,+49 1520 8865607,https://www.eversports.de/s/poleworkx,"[(Noch keine Bewertungen), Poledance]",None,2024-02-10 16:21:00,2024-02-10 16:21:00
2,pure body concepts OLD,"[Zugspitzstraße 2A, 85591 Vaterstetten]",85591,Vaterstetten,Zugspitzstraße 2A,[Übersicht],Diese Sportanlage bietet eine Vielzahl an Spor...,info@pure-body-concepts.de,None,+49 8106 3774499,https://www.eversports.de/s/pure-body-concepts,"[(Noch keine Bewertungen), Pilates, Tanzen, Yo...",None,2024-02-10 16:21:03,2024-02-10 16:21:03
3,Pole for Soul GbR,"[Die Schlagd 1 , 34125 Kassel]",34125,Kassel,Die Schlagd 1,"[Übersicht, Klassen, Workshops, Videos, Team]","Dein Studio für Poledance, Tanz, Aerial Hoop &...",info@poleforsoul.de,http://www.poleforsoul.com,0176 56826749,https://www.eversports.de/s/pole-for-soul-gbr,"[Poledance, Aerial Yoga, Poledance, Aerial Yog...",Zum Stundenplan,2024-02-10 16:21:17,2024-02-10 16:21:17
4,Pole Paradise,"[Ravensburgerstraße 110-112 , 50739 Köln]",50739,Köln,Ravensburgerstraße 110-112,"[Übersicht, Klassen, Workshops, Videos, Preise...","Das Pole Paradise in Köln ist ein Studio, in d...",poleparadise@gmx.de,https://www.pole-paradise.com,+491774616007,https://www.eversports.de/s/pole-paradise,"[Gymnastik, Turnen, Fitness, Gymnastik, Fitnes...",Zum Stundenplan,2024-02-10 16:21:31,2024-02-10 16:21:31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,The Pole Jungle,"[Volksgartenstraße 19, 4020 Linz]",4020,Linz,Volksgartenstraße 19,"[Übersicht, Klassen, Trainings, Kurse, Preise,...",Wie der Name bereits sagt erwartet euch ein gr...,hello@thepolejungle.com,http://www.thepolejungle.com,+43 677 647 33 8 00,https://www.eversports.de/s/the-pole-jungle,"[Poledance, Poledance, Poledance, Poledance, P...",Zum Stundenplan,2024-02-10 16:52:18,2024-02-10 16:52:18
156,Dayuma - Pole & Acrobatics,"[Auf dem Wolf 18 , 4052 Basel]",4052,Basel,Auf dem Wolf 18,"[Übersicht, Klassen, Workshops, Preise, Team]",Das aufgestellte Frauenteam legt grossen Wert ...,info@studio-dayuma.ch,http://www.studio-dayuma.ch,+41 76 537 39 99,https://www.eversports.de/s/dayuma-pole-and-ac...,"[Poledance, Akrobatik, Akrobatik, Poledance, A...",Kostenlos Schnuppern,2024-02-10 16:52:29,2024-02-10 16:52:29
157,POLEside,"[Saline 5 , 74177 Bad Friedrichshall]",74177,Bad,Saline 5,"[Übersicht, Klassen, Kurse, Workshops, Videos,...",Willkommen bei POLEside!Lass Dich von uns mitr...,info@poleside.de,http://poleside.de/,0176 870 191 81,https://www.eversports.de/s/poleside,"[Fitness, Aerial Yoga, Poledance, Poledance, A...",Zum Stundenplan,2024-02-10 16:52:49,2024-02-10 16:52:49
158,SWISS MEGA PARK,"[Schützenstrasse 2, 4402 Frenkendorf]",4402,Frenkendorf,Schützenstrasse 2,"[Übersicht, Sportplätze, Matches]",Der SWISS MEGA PARK in Frenkendorf ist nicht n...,kontakt@swissmegapark.ch,http://www.swissmegapark.ch/,+41 61 901 76 64,https://www.eversports.de/s/swiss-mega-park,"[12 Rezensionen, Tennis, Badminton, Basketball...",None,2024-02-10 16:52:56,2024-02-10 16:52:56


In [ ]:
result_workshop_details

,Workshop Name,Description,Studio Name,Location,Level,Date,URL_E,Time
0,Schnupperstunde Poledance,Trau dich und beginne jetzt! Damit du uns und ...,Pole for Soul GbR,13:30 - 14:30,Poledance,"Sonntag, 3. März 2024",https://www.eversports.de/e/workshop/rnbGVfh,13:30 - 14:30
1,Exotic Valentins Choreo,-Be my Exotic Valentine- Das ist das Motto für...,Pole Paradise,Ticket Abo Mitglieder,Tanzen,"Samstag, 17. Februar 2024",https://www.eversports.de/e/workshop/NLk1782,17:30 - 19:30
2,Stripped,"ohne Pole / alle LevelEgal, ob du deine:n Lieb...",Vertical - Pole Aerial Dance,Einzelticket,Poledance,"Samstag, 10. Februar 2024",https://www.eversports.de/e/workshop/mylmioi,17:00 - 18:30
3,Stripped,"ohne Pole / alle LevelEgal, ob du deine:n Lieb...",Vertical - Pole Aerial Dance,Einzelticket,Poledance,"Samstag, 10. Februar 2024",https://www.eversports.de/e/workshop/Yace1e3,19:00 - 20:30
4,Spagat,Straddle & Middlesplit stehen ganz oben auf De...,Luftbonbon,20:20 - 21:20,Stärkung/Flexibilität,"Begann am Montag, 5. Februar 2024",https://www.eversports.de/e/workshop/60kU5cU,"Königstraße 45, 32547 Bad Oeynhausen"
...,...,...,...,...,...,...,...,...
272,Lapdance Valentine's Special,Woman only Valentins Special! It's getting hot...,PoleCreation,Einzelticket,Fitness,"Samstag, 17. Februar 2024",https://www.eversports.de/e/workshop/bmZxua4,17:00 - 18:30
273,Lapdance Valentine's Special,Woman only Valentins Special! It's getting hot...,PoleCreation,Einzelticket,Fitness,"Samstag, 17. Februar 2024",https://www.eversports.de/e/workshop/qcRMhzd,18:45 - 20:15
274,Lapdance Valentine's Special,Woman only Valentins Special! It's getting hot...,PoleCreation,Einzelticket,Fitness,"Samstag, 24. Februar 2024",https://www.eversports.de/e/workshop/Zj0NETg,15:30 - 17:00
275,Kopfstand mal ganz einfach: FeetUp Workshop mi...,"Wer die Welt auf den Kopf stellt, dem liegt de...",Yoga Vidya Burgdorf,Ticketpreis für Abo-Mitglieder von YV Burgdorf,Yoga,"Samstag, 2. März 2024",https://www.eversports.de/e/workshop/0E9ASI_,14:00 - 17:00
